In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from pathlib import Path
import os
import pickle
from pathlib import Path
import os
from tensorflow import keras
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

plt.rcParams["figure.figsize"] = (30, 10)
plt.rcParams["figure.dpi"] = 600

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
TICKER = "TTWO"
MODEL_VERSION = 0.1
MODEL_NAME = "lstm"

In [3]:
models_path = Path.cwd() / "data" / "lstm" / "versions"

In [4]:
models_path

PosixPath('/home/ubuntu/projects/GAN-market-prediction/models/lstm/data/lstm/versions')

In [5]:
load_path = Path.cwd().parents[1] / "data" / "scaled_data"
models_path = Path.cwd() / "versions"

with open(load_path / f"data_{TICKER}.pickle", "rb") as handle:
    data = pickle.load(handle)

with open(load_path / f"scalers_{TICKER}.pickle", "rb") as handle:
    scalers = pickle.load(handle)

/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
data.keys()

dict_keys(['X_list_train', 'X_list_test', 'Y_preds_real_list_train', 'Y_preds_real_list_test', 'Y_whole_real_list_train', 'Y_whole_real_list_test'])

In [7]:
model = keras.models.load_model(models_path / f"lstm_{TICKER}_{MODEL_VERSION}")
preds = model.predict(data["X_list_test"])
backtest_path = Path(os.path.abspath("")).parents[1] / "backtesting"
actual_values = data["Y_preds_real_list_test"]
test_preds = scalers["y_scaler"].inverse_transform(preds)
test_true = scalers["y_scaler"].inverse_transform(actual_values)

6/6 [==============================] - 2s 4ms/step


In [8]:
mean_absolute_error(y_true=test_true, y_pred=test_preds)


7.286635838197858

In [9]:
mean_absolute_percentage_error(y_true=test_true, y_pred=test_preds)

0.043482908105509685

In [10]:
mean_squared_error(y_true=test_true, y_pred=test_preds) ** 0.5

8.78379421848192

In [11]:
import numpy as np
smape = np.mean(2 * np.abs(test_preds - test_true) / (np.abs(test_true) + np.abs(test_preds))) * 100
smape

4.250110309831089